In [0]:
file_path = "/Volumes/dev/sales_bronze/raw/orders/v1/Online-Store-Orders.csv"

df = (spark.read
      .format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load(file_path))

In [0]:
df.printSchema()

root
 |-- OrderID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- ShippingAddress: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- OrderStatus: string (nullable = true)
 |-- TrackingNumber: string (nullable = true)
 |-- ItemsInCart: integer (nullable = true)
 |-- CouponCode: string (nullable = true)
 |-- ReferralSource: string (nullable = true)
 |-- TotalPrice: double (nullable = true)



In [0]:
display(df)

OrderID,Date,CustomerID,Product,Quantity,UnitPrice,ShippingAddress,PaymentMethod,OrderStatus,TrackingNumber,ItemsInCart,CouponCode,ReferralSource,TotalPrice
ORD200000,2023-01-04,C72649,Monitor,5,570.62,928 Main St,Debit Card,Shipped,TRK37947903,7,SAVE10,Instagram,2853.1
ORD200001,2024-08-23,C75739,Phone,2,151.35,823 Main St,Online,Shipped,TRK91186779,3,SAVE10,Referral,302.7
ORD200002,2024-02-27,C81728,Tablet,5,550.68,512 Main St,Credit Card,Cancelled,TRK42903982,8,FREESHIP,Email,2753.4
ORD200003,2023-10-15,C33540,Chair,1,273.19,275 Main St,Debit Card,Returned,TRK62788070,5,SAVE10,Facebook,273.19
ORD200004,2025-05-08,C81840,Printer,4,626.01,668 Main St,Online,Delivered,TRK29241424,8,SAVE10,Email,2504.04
ORD200005,2023-10-23,C37249,Phone,2,245.86,934 Main St,Credit Card,Shipped,TRK72976927,4,SAVE10,Instagram,491.72
ORD200006,2025-06-17,C83492,Laptop,1,664.42,986 Main St,Gift Card,Returned,TRK96417362,6,SAVE10,Facebook,664.42
ORD200007,2023-05-12,C41460,Monitor,5,149.55,706 Main St,Cash,Shipped,TRK78809193,9,FREESHIP,Facebook,747.75
ORD200008,2025-04-02,C26817,Phone,2,134.28,904 Main St,Gift Card,Cancelled,TRK61042692,2,null,Email,268.56
ORD200009,2023-11-21,C31946,Desk,4,509.38,102 Main St,Credit Card,Shipped,TRK33478363,6,SAVE10,Google,2037.52


In [0]:
# Data Quality Checks

from pyspark.sql.functions import col, when, abs, current_date, sum as spark_sum

total_rows = df.count()
tolerance = 1e-3

null_counter = [spark_sum(when(col(col_name).isNull(), 1).otherwise(0)).alias(col_name)
                for col_name in df.columns if col_name != "CouponCode"]

null_counts_df = df.select(*null_counter).collect()[0]

for col_name in df.columns:
    if col_name != "CouponCode":
        null_count = null_counts_df[col_name]
        assert null_count == 0, f"Found {null_count} null values in column: {col_name}"

unique_OrderID_rows = df.select("OrderID").distinct().count()
assert unique_OrderID_rows == total_rows, "Found duplicate OrderID values"

incorrect_totalprices = df.filter(abs((col("Quantity") * col("UnitPrice")) - col("TotalPrice")) > tolerance).count()
assert incorrect_totalprices == 0, f"TotalPrice contains {incorrect_totalprices} incorrect values"

invalid_dates = df.filter((col("Date") > current_date()) | (col("Date") < "1900-01-01")).count()
assert invalid_dates == 0, f"Found {invalid_dates} invalid dates"


In [0]:
# Tranformations

from pyspark.sql.functions import avg, round, count

df = df.withColumn("UsedCoupon", when(col("CouponCode").isNotNull(), True).otherwise(False))

coupon_usage_df = df.groupBy("UsedCoupon").count()
coupon_usage_df = coupon_usage_df.withColumn("UsagePercentage", col("count") / total_rows)

aov_coupon_df = df.groupBy("UsedCoupon").agg(
    round(spark_sum("TotalPrice"), 2).alias("TotalRevenueUsingCoupons"),
    round(avg("TotalPrice"), 2).alias("AverageOrderValue")
)

referral_metrics_df = df.groupBy("ReferralSource").agg(
    round(spark_sum("TotalPrice"), 2).alias("TotalRevenue"),
    count("*").alias("TotalOrders"),
    round(avg(col("UsedCoupon").cast("int")), 4).alias("CouponUsageRate")
)

In [0]:
coupon_usage_df.show()
aov_coupon_df.show()
referral_metrics_df.show()

+----------+-----+---------------+
|UsedCoupon|count|UsagePercentage|
+----------+-----+---------------+
|      true|  891|         0.7425|
|     false|  309|         0.2575|
+----------+-----+---------------+

+----------+------------------------+-----------------+
|UsedCoupon|TotalRevenueUsingCoupons|AverageOrderValue|
+----------+------------------------+-----------------+
|      true|               942360.55|          1057.64|
|     false|               322401.41|          1043.37|
+----------+------------------------+-----------------+

+--------------+------------+-----------+---------------+
|ReferralSource|TotalRevenue|TotalOrders|CouponUsageRate|
+--------------+------------+-----------+---------------+
|        Google|   250441.48|        241|         0.7261|
|      Referral|   226815.58|        222|         0.7297|
|         Email|   261808.55|        250|           0.78|
|     Instagram|   275285.45|        259|         0.7645|
|      Facebook|    250410.9|        228|     

In [0]:
# Analytics

df.createOrReplaceTempView("silver_orders")

gold_referral_df = spark.sql("""
    SELECT
        ReferralSource,
        COUNT(*) AS TotalOrders,
        ROUND(SUM(TotalPrice), 2) AS TotalRevenue,
        SUM(CAST(UsedCoupon AS INT)) AS OrdersWithCoupon,
        ROUND(AVG(TotalPrice), 2) AS AverageOrderValue,
        ROUND(AVG(CAST(UsedCoupon AS INT)), 4) AS CouponUsageRate
    FROM silver_orders
    GROUP BY ReferralSource
    ORDER BY TotalRevenue DESC
""")

gold_referral_df.show()

+--------------+-----------+------------+----------------+-----------------+---------------+
|ReferralSource|TotalOrders|TotalRevenue|OrdersWithCoupon|AverageOrderValue|CouponUsageRate|
+--------------+-----------+------------+----------------+-----------------+---------------+
|     Instagram|        259|   275285.45|             198|          1062.88|         0.7645|
|         Email|        250|   261808.55|             195|          1047.23|           0.78|
|        Google|        241|   250441.48|             175|          1039.18|         0.7261|
|      Facebook|        228|    250410.9|             161|          1098.29|         0.7061|
|      Referral|        222|   226815.58|             162|          1021.69|         0.7297|
+--------------+-----------+------------+----------------+-----------------+---------------+

